In [1]:
import pandas as pd
from glob import glob

In [9]:
datasets = ["agnews", "sst-5", "gsm8k", "copa", "subj"]
models = ["llama", "qwen", "mistral"]
seeds = [42, 43, 44]
dfs = []
for dataset in datasets:
    for model in models:
        for seed in seeds:
            path = glob(f"../results/{dataset}/{model}/CAPO/seed{seed}/*/*/prompt_scores.parquet")[
                0
            ]
            df = pd.read_parquet(path)
            df["evaluated_blocks"] = 10 - df.isnull().sum(axis=1)
            df["evaluated_blocks_when_no_racing"] = 10
            dfs.append(
                df.assign(
                    dataset=dataset,
                    model=model,
                    seed=seed,
                )
            )

df = pd.concat(dfs, ignore_index=True)

In [13]:
df = df.groupby(["dataset", "model"]).agg(
    mean_evaluated_blocks=("evaluated_blocks", "sum"),
    mean_evaluated_blocks_when_no_racing=("evaluated_blocks_when_no_racing", "sum"),
)

In [14]:
df["saved_evals"] = df["mean_evaluated_blocks_when_no_racing"] - df["mean_evaluated_blocks"]
df["saved_evals_ratio"] = df["saved_evals"] / df["mean_evaluated_blocks_when_no_racing"]

In [15]:
df

mean_evaluated_blocks  mean_evaluated_blocks_when_no_racing  \
dataset model                                                                  
agnews  llama                     2787                                  5660   
        mistral                   1825                                  4070   
        qwen                      2121                                  3930   
copa    llama                     2414                                  5070   
        mistral                   2264                                  3820   
        qwen                      2846                                  4700   
gsm8k   llama                      953                                  1890   
        mistral                    942                                  1370   
        qwen                      1130                                  1900   
sst-5   llama                     2498                                  3950   
        mistral                   2110                                  3280   
        qwen                      2509                                  3210   
subj    llama                     1945                                  4700   
        mistral                   1875                                  3780   
        qwen                      2018                                  4080   

                 saved_evals  saved_evals_ratio  
dataset model                                    
agnews  llama           2873           0.507597  
        mistral         2245           0.551597  
        qwen            1809           0.460305  
copa    llama           2656           0.523866  
        mistral         1556           0.407330  
        qwen            1854           0.394468  
gsm8k   llama            937           0.495767  
        mistral          428           0.312409  
        qwen             770           0.405263  
sst-5   llama           1452           0.367595  
        mistral         1170           0.356707  
        qwen             701           0.218380  
subj    llama           2755           0.586170  
        mistral         1905           0.503968  
        qwen            2062           0.505392

In [17]:
df[["mean_evaluated_blocks", "mean_evaluated_blocks_when_no_racing", "saved_evals_ratio"]]

mean_evaluated_blocks  mean_evaluated_blocks_when_no_racing  \
dataset model                                                                  
agnews  llama                     2787                                  5660   
        mistral                   1825                                  4070   
        qwen                      2121                                  3930   
copa    llama                     2414                                  5070   
        mistral                   2264                                  3820   
        qwen                      2846                                  4700   
gsm8k   llama                      953                                  1890   
        mistral                    942                                  1370   
        qwen                      1130                                  1900   
sst-5   llama                     2498                                  3950   
        mistral                   2110                                  3280   
        qwen                      2509                                  3210   
subj    llama                     1945                                  4700   
        mistral                   1875                                  3780   
        qwen                      2018                                  4080   

                 saved_evals_ratio  
dataset model                       
agnews  llama             0.507597  
        mistral           0.551597  
        qwen              0.460305  
copa    llama             0.523866  
        mistral           0.407330  
        qwen              0.394468  
gsm8k   llama             0.495767  
        mistral           0.312409  
        qwen              0.405263  
sst-5   llama             0.367595  
        mistral           0.356707  
        qwen              0.218380  
subj    llama             0.586170  
        mistral           0.503968  
        qwen              0.505392

In [16]:
df["saved_evals_ratio"].mean()

0.4397876950982689